# Daru::DataFrame の使い方

ここではDaruのデータフレームの使い方を紹介します。

In [1]:
require 'daru'

true

## 基本的なデータフレーム作成方法とデータへのアクセス方法

### 配列の配列からのデータフレーム作成

配列の配列からのデータフレーム作成は下記になります。*index* はオプショナルで無ければ0始まりの数字が付与されます。


In [2]:
df = Daru::DataFrame.new([[1,2,3,4], [1,2,3,4]],order: [:a, :b])

#<Daru::DataFrame(4x2)>
       a   b
   0   1   1
   1   2   2
   2   3   3
   3   4   4

In [3]:
df = Daru::DataFrame.new([[1,2,3,4], [1,2,3,4]],order: [:a, :b], index: [:one, :two, :three, :four])

#<Daru::DataFrame(4x2)>
           a     b
   one     1     1
   two     2     2
 three     3     3
  four     4     4

### 配列のハッシュからのデータフレーム作成

最も自然なデータフレーム作成方法はハッシュからになるでしょう。下記のように行います。

In [4]:
df = Daru::DataFrame.new({a: [1,2,3,4], b: [1,2,3,4]},order: [:b, :a])

#<Daru::DataFrame(4x2)>
       b   a
   0   1   1
   1   2   2
   2   3   3
   3   4   4

### DaruのVectorをつなげることによるデータフレーム作成
DaruのVecterをつなげることでもデータフレームを作ることが可能です。

In [5]:
v1 = Daru::Vector.new([1,2,3,4,5], index: [:a, :b, :c, :d, :e])
v2 = Daru::Vector.new([11,22,33,44], index: [:b, :e, :a, :absent])

Daru::DataFrame.new({v1: v1, v2: v2})

#<Daru::DataFrame(6x2)>
            v1     v2
      a      1     33
 absent    nil     44
      b      2     11
      c      3    nil
      d      4    nil
      e      5     22

### 行からの作成

これまでは列指向でデータフレームを作ってきましたが行指向で作ることも可能です。
その場合は`rows`メソッドを使います。


In [6]:
Daru::DataFrame.rows([
  [1,11,10,'a'],
  [2,22,20 ,4 ],
  [3,33,30,'g'],
  [4,44,40, 3 ]
  ], order: [:a, :b, :c, :d])

#<Daru::DataFrame(4x4)>
       a   b   c   d
   0   1  11  10   a
   1   2  22  20   4
   2   3  33  30   g
   3   4  44  40   3

`rows`メソッドを使う場合も`index`が使えます。

In [7]:
r1 = Daru::Vector.new([1,2,3,4,5], index: [:a, :b, :c, :d, :e])
r2 = Daru::Vector.new([11,22,33,44,55], index: [:a, :c, :e, :b, :odd])

Daru::DataFrame.rows([r1,r2], order: [:a, :b, :c, :d, :odd])

#<Daru::DataFrame(2x5)>
       a   b   c   d odd
   0   1   2   3   4 nil
   1  11  44  22 nil  55

## CSV や Excel の読み込み

### CSV の読み込み

は下記で可能です。

In [8]:
Daru::DataFrame.from_csv 'data/sales-funnel.csv'

#<Daru::DataFrame(17x8)>
               Account       Name        Rep    Manager    Product   Quantity      Price     Status
          0     714466 Trantow-Ba Craig Book Debra Henl        CPU          1      30000  presented
          1     714466 Trantow-Ba Craig Book Debra Henl   Software          1      10000  presented
          2     714466 Trantow-Ba Craig Book Debra Henl Maintenanc          2       5000    pending
          3     737550 Fritsch, R Craig Book Debra Henl        CPU          1      35000   declined
          4     146832 Kiehn-Spin Daniel Hil Debra Henl        CPU          2      65000        won
          5     218895  Kulas Inc Daniel Hil Debra Henl        CPU          2      40000    pending
          6     218895  Kulas Inc Daniel Hil Debra Henl   Software          1      10000  presented
          7     412290 Jerde-Hilp John Smith Debra Henl Maintenanc          2       5000    pending
          8     740150 Barton LLC John Smith Debra Henl        CPU          1      35000   declined
          9     141962 Herman LLC Cedric Mos Fred Ander        CPU          2      65000        won
         10     163416 Purdy-Kund Cedric Mos Fred Ander        CPU          1      30000  presented
         11     239344 Stokes LLC Cedric Mos Fred Ander Maintenanc          1       5000    pending
         12     239344 Stokes LLC Cedric Mos Fred Ander   Software          1      10000  presented
         13     307599 Kassulke,  Wendy Yule Fred Ander Maintenanc          3       7000        won
         14     688981 Keeling LL Wendy Yule Fred Ander        CPU          5     100000        won
        ...        ...        ...        ...        ...        ...        ...        ...        ...

### Excel ファイルの読み込み

も下記で可能です。
裏では[spreadsheet](https://github.com/zdavatz/spreadsheet) gemを利用しています。


In [9]:
df = Daru::DataFrame.from_excel 'data/test_xls.xls'

#<Daru::DataFrame(6x5)>
                id     name      age     city       a1
        0        1     Alex       20 New York      a,b
        1        2   Claude       23   London      b,c
        2        3    Peter       25   London        a
        3        4    Franz      nil    Paris      nil
        4        5   George      5.5     Tome    a,b,c
        5        6  Fernand      nil      nil      nil

## データのクエリとアクセス

`Daru::DataFrame`は行とベクトルで構成され、どちらも直感的な構文を使ってラベルからアクセスできます。

次のDataFrameを考えてみましょう。

In [10]:
df = Daru::DataFrame.new({
  a: [1,2,3,4,5,6,7], 
  b: ['a','b','c','d','e','f','g'], 
  c: [11,22,33,44,55,66,77]
  }, index: [:a,:b,:c,:d,:e,:f,:g])

#<Daru::DataFrame(7x3)>
       a   b   c
   a   1   a  11
   b   2   b  22
   c   3   c  33
   d   4   d  44
   e   5   e  55
   f   6   f  66
   g   7   g  77

`#[]`演算子を使用して任意のVectorにアクセスできます。
結果のVectorは、DataFrameのインデックスを保持する`Daru::Vector`として返されます。


In [11]:
df[:b]

#<Daru::Vector(7)>
       b
   a   a
   b   b
   c   c
   d   d
   e   e
   f   f
   g   g

`#[]`内で範囲を指定して、範囲内の列を含むDataFrameを返すこともできます。


In [12]:
df[:b..:c]

#<Daru::DataFrame(7x2)>
       b   c
   a   a  11
   b   b  22
   c   c  33
   d   d  44
   e   e  55
   f   f  66
   g   g  77

行は`#row[]`メソッドを使用してアクセスできます。

返される行のインデックスは、Vectorの名前に対応します。

In [13]:
df.row[:c]

#<Daru::Vector(3)>
       c
   a   3
   b   c
   c  33

ここでも範囲を指定することができ、範囲で指定された関連行を含む`Daru::DataFrame`を`Daru::Vector`の代わりに受け取ります。


In [14]:
df.row[:d..:f]

#<Daru::DataFrame(3x3)>
       a   b   c
   d   4   d  44
   e   5   e  55
   f   6   f  66

行には数値インデックスを使用してアクセスすることもできます（これは列に対しても機能します）。


In [15]:
df.row[3]

#<Daru::Vector(3)>
       3
   a   4
   b   d
   c  44

`#head`メソッドに引数を渡して、上位3行を取得できます（または`#tail`を使用して下の3行）。


In [16]:
df.head 3

#<Daru::DataFrame(3x3)>
       a   b   c
   a   1   a  11
   b   2   b  22
   c   3   c  33

## データのフィルタリング、選択、追加、削除

`#[] =`演算子を使用して名前と値を指定するだけで、列を追加できます。


In [17]:
df[:d] = df[:a] * df[:c]
df

#<Daru::DataFrame(7x4)>
       a   b   c   d
   a   1   a  11  11
   b   2   b  22  44
   c   3   c  33  99
   d   4   d  44 176
   e   5   e  55 275
   f   6   f  66 396
   g   7   g  77 539

`#delete_vector`メソッドでベクトルを削除できます。


In [18]:
df.delete_vector :b

#<Daru::DataFrame(7x3)>
       a   c   d
   a   1  11  11
   b   2  22  44
   c   3  33  99
   d   4  44 176
   e   5  55 275
   f   6  66 396
   g   7  77 539

DataFrameのインデックスに適合しない`Daru::Vector`を挿入しようとすると、値はDataFrameのインデックスに適合するように適切に配置されます。

DataFrame上に類似のインデックスが見つからない場合は、`nil`が挿入されます。

配列を挿入するには、配列の長さをDataFrameの長さと同じにする必要があります。


In [19]:
df[:b] = Daru::Vector.new(['a',33,'b','c','d',88,'e'], index: [:a,:c,:d,:b,:e,:f,:extra])
df

#<Daru::DataFrame(7x4)>
       a   c   d   b
   a   1  11  11   a
   b   2  22  44   c
   c   3  33  99  33
   d   4  44 176   b
   e   5  55 275   d
   f   6  66 396  88
   g   7  77 539 nil

行を挿入する場合も同様です。

In [20]:
df.row[:latest] = Daru::Vector.new([10,20,30,40], index: [:c,:b,:a,:d])
df

#<Daru::DataFrame(8x4)>
             a      c      d      b
      a      1     11     11      a
      b      2     22     44      c
      c      3     33     99     33
      d      4     44    176      b
      e      5     55    275      d
      f      6     66    396     88
      g      7     77    539    nil
 latest     30     10     40     20

行とベクトルの両方の挿入で、指定されたインデックスがDataFrameに存在しない場合は、新しいインデックスが作成され、追加されます。存在する場合、既存のインデックスは上書きされます。


特定の行/ベクトルをその値に基づいてフィルタリングするには、`#filter`メソッドを使用します。
デフォルトでは、ベクトルを反復処理し、ブロックがtrueを返すベクトルを保持します。
オプションには_axis_引数を受け取ります。
この引数を使用すると、ベクトルまたは行を反復処理するかどうかを指定できます。


In [21]:
# Filter vectors.

# The `type` method returns either :numeric or :object. The :numeric type states
# that the Vector consists only of numerical data (combined with missing data).
# If the type happens to be :object, it contains non-numerical data like strings
# or symbols. Statistical operations will not be possible on Vectors of type :object.

df.filter do |vector|
  vector.type == :numeric and vector.median < 50
end

#<Daru::DataFrame(8x2)>
             a      c
      a      1     11
      b      2     22
      c      3     33
      d      4     44
      e      5     55
      f      6     66
      g      7     77
 latest     30     10

In [22]:
# Filter rows

df.filter(:row) do |row|
  row[:a] + row[:d] < 100
end

#<Daru::DataFrame(3x4)>
             a      c      d      b
      a      1     11     11      a
      b      2     22     44      c
 latest     30     10     40     20

DataFrameは、`#transpose`メソッドを使用して転置することができます。


In [23]:
df.transpose

#<Daru::DataFrame(4x8)>
             a      b      c      d      e      f      g latest
      a      1      2      3      4      5      6      7     30
      c     11     22     33     44     55     66     77     10
      d     11     44     99    176    275    396    539     40
      b      a      c     33      b      d     88    nil     20

## 算術演算について

すべての算術演算は`Daru::DataFrame`で実行でき、別のDataFrame、Vector、またはスカラを持つDataFrameを使用できます。

非スカラー量で操作が実行されるたびに、インデックスは適切に整列されます。

**スカラを使うもの**

スカラ量を追加すると、その数値がすべての数値型ベクトルに追加され、オブジェクトタイプのベクトルが元のとおりに保持されます。


In [24]:
df + 10

#<Daru::DataFrame(8x4)>
             a      c      d      b
      a     11     21     21      a
      b     12     32     54      c
      c     13     43    109     33
      d     14     54    186      b
      e     15     65    285      d
      f     16     76    406     88
      g     17     87    549    nil
 latest     40     20     50     20

** 他のDataFrameとの演算 **

2つのデータフレーム間で算術演算を実行すると、いずれかのデータフレームの行および列のインデックスによって要素が整列されます。

一方のデータフレームには列が存在し、他方の列には列が存在しない場合、その結果のデータフレームには、その名前の`nils`で完全な列が設定されます。

この操作が成功するには、DataFramesのサイズが同じである必要はありません。

In [25]:
df1 = Daru::DataFrame.new({
  a: 7.times.map { rand(100) },
  f: 7.times.map { rand(100) },
  c: 7.times.map { rand(100) }
  }, index: [:a,:b,:c,:d,:latest,:older,:f])

df1 + df

#<Daru::DataFrame(9x5)>
             a      b      c      d      f
      a     69    nil    105    nil    nil
      b     29    nil     76    nil    nil
      c     57    nil     75    nil    nil
      d     57    nil    130    nil    nil
      e    nil    nil    nil    nil    nil
      f     56    nil    135    nil    nil
      g    nil    nil    nil    nil    nil
 latest     62    nil     67    nil    nil
  older    nil    nil    nil    nil    nil

## Statistics

統計演算では、数値ベクトルのみの基本的な統計計算を実行します。

メソッドの全リストについては、ドキュメントの`Daru::Maths::Statistics::DataFrame`モジュールを参照してください。
https://rubygems.org/gems/daru

デモを行うには、`#mean`メソッドは各数値ベクトルの平均を計算し、対応する値と共にインデックスとしてのベクトルの名前を持つ`Daru::Vector`を返します。


In [26]:
df.mean

#<Daru::Vector(3)>
        mean
     a  7.25
     c 39.75
     d 197.5

`#describe`メソッドを使用すると、さまざまな統計値をワンショットで知ることができます。


In [27]:
df.describe

#<Daru::DataFrame(5x3)>
                     a          c          d
      count          8          8          8
       mean       7.25      39.75      197.5
        std 9.40744386 25.0699022 190.992146
        min          1         10         11
        max         30         77        539

`#cov`はDataFrameの共分散行列を返し、適切にインデックスが作成されるので、データをはっきりと見ることができます。


In [28]:
df.cov

#<Daru::DataFrame(3x3)>
               a       c       d
       a    88.5   -66.5  -233.0
       c   -66.5   628.5  4637.0
       d  -233.0  4637.0 36478.0

同様に`#corr`は相関行列を計算します。


In [29]:
df.corr

#<Daru::DataFrame(3x3)>
                     a          c          d
          a        1.0 -0.2819664 -0.1296787
          c -0.2819664 0.99999999 0.96843158
          d -0.1296787 0.96843158        1.0

## ループとイテレータ

`Daru::DataFrame`には、行または列のループ処理を行うためのイテレータが多数用意されています。

**#each**

`#each`は`Array#`とまったく同じように動作します。
それぞれのデフォルトモードは、DataFrameの**列**に対して反復処理を行うモードです。
**行**を反復処理するには、`axis`を渡す必要があります。
つまり、`:row`を引数として渡す必要があります。


In [30]:
# Iterate over vectors

e = []
df.each do |vector|
  e << vector[:a].to_s + vector[:latest].to_s
end

puts e

["130", "1110", "1140", "a20"]


In [31]:
# Iterate over rows

r = []
df.each(:row) do |row|
  r << row[:a] * row[:c]
end

puts r

[11, 44, 99, 176, 275, 396, 539, 300]


**#map**

`#map`イテレータは`Array#map`のように動作します。
ブロックの各実行によって返された値が配列に追加され、配列が返されます。

このメソッドは、`#each`のような`:axis`の引数も受け付けます。
デフォルトは`:vector`です。

In [32]:
# Map over vectors. 

# The `only_numerics` method returns a DataFrame which contains vectors 
# with only numerical values. Setting the `:clone` option to false will 
# return the same Vector objects that are contained in the original DataFrame.

df.only_numerics(clone: false).map do |vector| 
  vector.mean
end

[7.25, 39.75, 197.5]

In [33]:
# Map over rows.

# Calling `only_numerics` on a Daru::Vector will return a Vector with only numeric and
# missing data. Data marked as 'missing' is not considered during statistical computation.

df.map(:row) do |row|
  row.only_numerics.mean
end

[7.666666666666667, 22.666666666666668, 42.0, 74.66666666666667, 111.66666666666667, 139.0, 207.66666666666666, 25.0]

**#recode**

Recodeは`#map`と同様に動作しますが、2つの重要な違いは、recodeが配列の代わりに変更された`Daru::DataFrame`を返すことです。
このため、`#recode`はブロックの実行ごとに`Daru::Vector`を返すようにします。

`map`と同様に、recodeはオプショナルで`axis`引数も受け入れます。


In [34]:
# Recode vectors

df.only_numerics(clone: false).recode do |vector|
  vector[:a] = vector[:d]  + vector[:c]
  vector[:b] = vector.mean + vector[:a]
  vector # <- return the vector to the block
end

#<Daru::DataFrame(8x3)>
             a      c      d
      a      7     77    275
      b   15.0  125.0  505.5
      c      3     33     99
      d      4     44    176
      e      5     55    275
      f      6     66    396
      g      7     77    539
 latest     30     10     40

In [35]:
# Recode rows

df.recode(:row) do |row|
  row[:a] = row[:c] - row[:d]
  row[:b] = row[:b].to_i if row[:b].is_a?(String)
  row
end

#<Daru::DataFrame(8x4)>
             a      c      d      b
      a      0     11     11      0
      b    -22     22     44      0
      c    -66     33     99     33
      d   -132     44    176      0
      e   -220     55    275      0
      f   -330     66    396     88
      g   -462     77    539    nil
 latest    -30     10     40     20

**#collect**

`#collect`イテレータは`#map`に似ていますが、唯一の違いは、各ブロック実行の結果で構成される`Daru::Vector`が返されることです。
結果のVectorは、`collect`が反復された`axis`のインデックスと同じインデックスを持ちます。

オプションの _axis_ 引数も受け入れます。


In [36]:
# Collect Vectors

df.collect do |vector|
  vector[:c] + vector[:f]
end

#<Daru::Vector(4)>
   a   9
   c  99
   d 495
   b 121

In [37]:
# Collect Rows

df.collect(:row) do |row|
  row[:a] + row[:d] - row[:c]
end

#<Daru::Vector(8)>
      a      1
      b     24
      c     69
      d    136
      e    225
      f    336
      g    469
 latest     60

**#vector_by_calculation**

`#vector_by_calculation`は、ブロックによって返された結果に基づいて`Daru::Vector`を生成するために使用できるDSLです。

このDSLでは、要素をブロック内のメソッドとして直接参照できます。

In [38]:
df.vector_by_calculation { a + c + d }

#<Daru::Vector(8)>
      a     23
      b     68
      c    135
      d    224
      e    335
      f    468
      g    623
 latest     80

## ソートについて

`Daru::DataFrame`は、DataFrame内のVectorを階層的にソートするために使用できる堅牢な `＃sort`関数を提供します。

多くのオプションを実証するための例がいくつかあります：


In [39]:
df = Daru::DataFrame.new({
  a: ['g', 'g','g','sort', 'this'],
  b: [4,4,335,32,11],
  c: ['This', 'dataframe','is','for','sorting']
  })

#<Daru::DataFrame(5x3)>
                   a         b         c
         0         g         4      This
         1         g         4 dataframe
         2         g       335        is
         3      sort        32       for
         4      this        11   sorting

「sort」の引数として渡された配列は、各Vectorにソートの優先順位を与える順序をメソッドに指示します。

** :ascending **オプションは、DataFrameにVectorの並べ替え順序を指示します。
(昇順ソートの場合は*true* 、降順ソートの場合は*false*)

** :by **オプションを使用すると、並べ替える各Vectorのカスタム属性を定義できます。
これは、ブロックを`Array#sort_by`に渡すのと同様に機能します。


In [40]:
df.sort([:a,:b,:c], ascending: [true, false, true], by: {c: lambda { |a| a.size }})

#<Daru::DataFrame(5x3)>
                   a         b         c
         2         g       335        is
         0         g         4      This
         1         g         4 dataframe
         3      sort        32       for
         4      this        11   sorting

#### その他の例

VectorのシーケンスでDataFrameをソートしてみます。


In [41]:
df = Daru::DataFrame.new({a: [1,2,1,2,3], b: [5,4,3,2,1]})

df.sort [:a, :b]

#<Daru::DataFrame(5x2)>
       a   b
   2   1   3
   0   1   5
   3   2   2
   1   2   4
   4   3   1

ブロックなしでDataFrameをソートしてみましょう。
ここで`nils`は自動的に処理され、一番上に表示されます。

In [42]:
df = Daru::DataFrame.new({a: [-3,nil,-1,nil,5], b: [4,3,2,1,4]})

df.sort([:a])

#<Daru::DataFrame(5x2)>
       a   b
   1 nil   3
   3 nil   1
   0  -3   4
   2  -1   2
   4   5   4

`nils` が自動的に処理されたブロックでデータフレームをソートします。


In [43]:
df = Daru::DataFrame.new({a: [nil,-1,1,nil,-1,1], b: ['aaa','aa',nil,'baaa','x',nil] })

# df.sort [:b], by: {b: lambda { |a| a.length } }
# This would give "NoMethodError: undefined method `length' for nil:NilClass"

# Instead you could do the following if you want the nils to be handled automatically
df.sort [:b], by: {b: lambda { |a| a.length } }, handle_nils: true

#<Daru::DataFrame(6x2)>
         a    b
    2    1  nil
    5    1  nil
    4   -1    x
    1   -1   aa
    0  nil  aaa
    3  nil baaa

`nils`が手動で処理されたブロックでデータフレームをソートします。


In [44]:
df = Daru::DataFrame.new({a: [nil,-1,1,nil,-1,1], b: ['aaa','aa',nil,'baaa','x',nil] })

# To print nils at the bottom one can use lambda { |a| (a.nil?)[1]:[0,a.length] }
df.sort [:b], by: {b: lambda { |a| (a.nil?)?[1]:[0,a.length] } }, handle_nils: true

#<Daru::DataFrame(6x2)>
         a    b
    4   -1    x
    1   -1   aa
    0  nil  aaa
    3  nil baaa
    2    1  nil
    5    1  nil